In [1]:
#Importamos las librerías que vamos a necesitar
import pandas as pd
import numpy as np
import sys
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Librería para manejar archivos .env, para cargar tokens y claves
import os
import dotenv
dotenv.load_dotenv()

True

In [3]:
# Librería para trabajar con bases de datos SQL
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors

In [4]:
df_tablas = pd.read_csv(f"../datos/df_supermercados.csv")

## Preparación de las tablas para la base de datos

### Tabla productos

In [5]:
df_productos = pd.DataFrame(df_tablas["categoria_producto"].unique())
df_productos.reset_index(inplace=True)
df_productos.rename(columns={'index':'id_producto', 0:'nombre'}, inplace=True)
df_productos["id_producto"]=df_productos["id_producto"].apply(lambda x: x+1)
df_productos

,id_producto,nombre
0,1,aceite de girasol
1,2,aceite de oliva
2,3,leche


In [6]:
# df_productos.to_csv("../datos/tabla_productos.csv")

In [7]:
lista_tuplas_productos = []
for fila in df_productos.values:
    lista_tuplas_productos.append(tuple(fila))

### Tabla supermercados

In [8]:
df_superm = pd.DataFrame(df_tablas["supermercado"].unique())
df_superm.reset_index(inplace=True)
df_superm.rename(columns={'index':'id_super', 0:'nombre'}, inplace=True)
df_superm["id_super"]=df_superm["id_super"].apply(lambda x: x+1)
df_superm

,id_super,nombre
0,1,alcampo
1,2,carrefour
2,3,dia
3,4,eroski
4,5,hipercor
5,6,mercadona


In [9]:
# df_superm.to_csv("../datos/tabla_superm.csv")

In [10]:
lista_tuplas_supermercados = []
for fila in df_superm.values:
    lista_tuplas_supermercados.append(tuple(fila))

### Tabla histórico

Incluimos los id del supermercado y producto que serán las claves foraneas:

In [11]:
df_aux_historico = df_tablas.merge(right=df_superm, how='left', left_on ='supermercado', right_on="nombre")
df_aux_historico=df_aux_historico.drop(columns=["Unnamed: 0", "supermercado", "nombre"])
df_aux_historico_2 = df_aux_historico.merge(right=df_productos, how='left', left_on ='categoria_producto', right_on="nombre")
df_aux_historico_2 = df_aux_historico_2.drop(columns=["categoria_producto", "nombre"])

Reordenamos y borramos duplicados

In [12]:
df_aux_historico_2 = df_aux_historico_2.reindex(columns=['id_super', 'id_producto', 'nombre_producto', 'subcategoria', 'Día', 'Precio (€)', 'cantidad_variacion', 'porcentaje_variacion'])
df_aux_historico_2.drop_duplicates(inplace=True)
df_historico = df_aux_historico_2
df_historico.sample(5)

,id_super,id_producto,nombre_producto,subcategoria,Día,Precio (€),cantidad_variacion,porcentaje_variacion
83201,3,3,alcampo cultivamos lo bueno leche de vaca ente...,entera,2024-08-13,1.29,0.0,0.0
73783,3,2,fruto del sur aceite de oliva virgen extra bot...,virgen extra,2024-07-04,9.17,0.0,0.0
172780,6,2,carbonell aceite de oliva virgen extra spray 2...,virgen extra,2024-10-21,3.81,0.0,0.0
28467,1,3,pascual leche entera de vaca 1 l,entera,2024-07-24,1.22,0.0,0.0
57367,2,3,kaiku leche de vaca semidesnatada y sin lactos...,semidesnatada,2024-08-17,1.29,0.0,0.0


In [13]:
# df_historico.to_csv("../datos/tabla_historico.csv")

In [14]:
lista_tuplas_historico = []
for fila in df_historico.values:
    lista_tuplas_historico.append(tuple(fila))

In [15]:
#Guardamos de forma segura la contraeña para conectarnos a la base de datos.
key = os.getenv("token")

In [17]:
def conexion_a_dbeaver(database):
    """
    Establece una conexión a una base de datos DBeaver.

    Args:
        database (str): El nombre de la base de datos.

    Returns:
        conexion: Un objeto de conexión a la base de datos.
    """
    try:
        conexion = psycopg2.connect(
            database=database,
            user="postgres",
            password="admin",
            host="localhost",
            port="5432"
        )
    except OperationalError as e:
        if e.pgcode == errorcodes.INVALID_PASSWORD:
            print("La contraseña intorducida es errónea")
        elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
            print("Se ha producido un error de conexión")
        else:
            print(f"Ocurrió el error {e}")
    cursor = conexion.cursor()
    return conexion